# Reference data accuracy assessment by Radiant Earth

Radiant Earth is conducting an accuracy assessment of DE Africa cropmask reference data using the airbus high-res satellite archive. This notebook produces a confusion matrix between DE AFrica's labels and Radiant Earth's labels.  

Inputs will be:

1. `<AEZ-region_RE_sample_validation.geojson>` : The results from collecting training data in the CEO tool

Output will be:
1. A `confusion error matrix` containing Overall, Producer's, and User's accuracy, along with the F1 score.

***

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Analysis Parameters

In [2]:
folder = 'data/training_validation/collect_earth/central/'
gjson =  'data/training_validation/collect_earth/central/Central_region_RE_sample_validated.geojson'

### Load the dataset

In [3]:
#ground truth shapefile
df = gpd.read_file(gjson)
df.head()

,smpl_sampleid,smpl_class,Class,Validation_Class,geometry
0,394,1,non-crop,non-crop,"POLYGON ((28.78370 0.66496, 28.78412 0.66496, ..."
1,289,2,non-crop,non-crop,"POLYGON ((14.56301 10.86910, 14.56343 10.86910..."
2,799,1,non-crop,non-crop,"POLYGON ((18.85013 5.99825, 18.85055 5.99825, ..."
3,446,1,non-crop,non-crop,"POLYGON ((27.69872 1.29180, 27.69913 1.29180, ..."
4,992,1,non-crop,non-crop,"POLYGON ((19.91086 -15.80279, 19.91128 -15.802..."


### Clean up dataframe


In [4]:
#rename columns
df = df.rename(columns={'Class':'Prediction',
                        'Validation_Class':'Actual'})
df.head()

,smpl_sampleid,smpl_class,Prediction,Actual,geometry
0,394,1,non-crop,non-crop,"POLYGON ((28.78370 0.66496, 28.78412 0.66496, ..."
1,289,2,non-crop,non-crop,"POLYGON ((14.56301 10.86910, 14.56343 10.86910..."
2,799,1,non-crop,non-crop,"POLYGON ((18.85013 5.99825, 18.85055 5.99825, ..."
3,446,1,non-crop,non-crop,"POLYGON ((27.69872 1.29180, 27.69913 1.29180, ..."
4,992,1,non-crop,non-crop,"POLYGON ((19.91086 -15.80279, 19.91128 -15.802..."


***

### Reclassify prediction & actual columns

1 = crop, 
0 = non-crop

In [5]:
df['Prediction'] = np.where(df['Prediction']=='non-crop', 0, df['Prediction'])
df['Prediction'] = np.where(df['Prediction']=='crop', 1, df['Prediction'])

df['Actual'] = np.where(df['Actual']=='non-crop', 0, df['Actual'])
df['Actual'] = np.where(df['Actual']=='crop', 1, df['Actual'])

df.head()

,smpl_sampleid,smpl_class,Prediction,Actual,geometry
0,394,1,0,0,"POLYGON ((28.78370 0.66496, 28.78412 0.66496, ..."
1,289,2,0,0,"POLYGON ((14.56301 10.86910, 14.56343 10.86910..."
2,799,1,0,0,"POLYGON ((18.85013 5.99825, 18.85055 5.99825, ..."
3,446,1,0,0,"POLYGON ((27.69872 1.29180, 27.69913 1.29180, ..."
4,992,1,0,0,"POLYGON ((19.91086 -15.80279, 19.91128 -15.802..."


### Generate a confusion matrix with all classes

In [6]:
confusion_matrix = pd.crosstab(df['Actual'],
                               df['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,All
Actual,,,
0,25,1,26
1,0,26,26
N/A,4,0,4
mixed,1,3,4
All,30,30,60


### Reclassify into a binary assessment

In [7]:
counts = df.groupby('Actual').count()

print("Total number of samples: " + str(len(df)))
print("Number of 'mixed' samples: "+ str(counts[counts.index=='mixed']['Prediction'].values[0]))
# print("Number of 'N/A' samples: "+ str(counts[counts.index=='N/A']['Prediction'].values[0]))

print("Dropping 'mixed' and 'N/A' samples")

df = df.drop(df[df['Actual']=='mixed'].index)
df = df.drop(df[df['Actual']=='N/A'].index)

Total number of samples: 60
Number of 'mixed' samples: 4
Dropping 'mixed' and 'N/A' samples


---

### Recreate confusion matrix

In [8]:
confusion_matrix = pd.crosstab(df['Actual'],
                               df['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,All
Actual,,,
0,25,1,26
1,0,26,26
All,25,27,52


### Calculate User's and Producer's Accuracy

`Producer's Accuracy`

In [9]:
confusion_matrix["Producer's"] = [confusion_matrix.loc[0, 0] / confusion_matrix.loc[0, 'All'] * 100,
                              confusion_matrix.loc[1, 1] / confusion_matrix.loc[1, 'All'] * 100,
                              np.nan]

`User's Accuracy`

In [10]:
users_accuracy = pd.Series([confusion_matrix[0][0] / confusion_matrix[0]['All'] * 100,
                                confusion_matrix[1][1] / confusion_matrix[1]['All'] * 100]
                         ).rename("User's")

confusion_matrix = confusion_matrix.append(users_accuracy)

`Overall Accuracy`

In [11]:
confusion_matrix.loc["User's","Producer's"] = (confusion_matrix.loc[0, 0] + 
                                                confusion_matrix.loc[1, 1]) / confusion_matrix.loc['All', 'All'] * 100

`F1 Score`

The F1 score is the harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall), and is calculated as:

$$
\begin{aligned}
\text{Fscore} = 2 \times \frac{\text{UA} \times \text{PA}}{\text{UA} + \text{PA}}.
\end{aligned}
$$

Where UA = Users Accuracy, and PA = Producer's Accuracy

In [12]:
fscore = pd.Series([(2*(confusion_matrix.loc["User's", 0]*confusion_matrix.loc[0, "Producer's"]) / (confusion_matrix.loc["User's", 0]+confusion_matrix.loc[0, "Producer's"])) / 100,
                    f1_score(df['Actual'].astype(np.int8), df['Prediction'].astype(np.int8), average='binary')]
                         ).rename("F-score")

confusion_matrix = confusion_matrix.append(fscore)

### Tidy Confusion Matrix

* Limit decimal places,
* Add readable class names
* Remove non-sensical values 

In [13]:
# round numbers
confusion_matrix = confusion_matrix.round(decimals=2)

In [14]:
# rename booleans to class names
confusion_matrix = confusion_matrix.rename(columns={0:'Non-crop', 1:'Crop', 'All':'Total'},
                                            index={0:'Non-crop', 1:'Crop', 'All':'Total'})

In [15]:
#remove the nonsensical values in the table
confusion_matrix.loc["User's", 'Total'] = '--'
confusion_matrix.loc['Total', "Producer's"] = '--'
confusion_matrix.loc["F-score", 'Total'] = '--'
confusion_matrix.loc["F-score", "Producer's"] = '--'

In [16]:
confusion_matrix

Prediction,Non-crop,Crop,Total,Producer's
Actual,,,,
Non-crop,25.00,1.00,26,96.15
Crop,0.00,26.00,26,100
Total,25.00,27.00,52,--
User's,100.00,96.30,--,98.08
F-score,0.98,0.98,--,--


### Export csv

In [17]:
# confusion_matrix.to_csv(folder+ 'radiant_earth_reference_data_accuracy_results.csv')